## LIMPIEZA DE DATOS ARCHIVOS CSV:


(Pendiente explicación de lo que contienen los archivos)

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('../src')
from unidecode import unidecode
pd.set_option('display.max_columns', None) 

### LIMPIEZA ARCHIVO DENUNCIAS POR VIOLENCIA DE GÉNERO:


In [2]:
denu = pd.read_csv('../data/denuncias.csv')
denu.head()

,Año,Trimestre,Origen de la denuncia,Almería,Cádiz,Córdoba,Granada,Huelva,Jaén,Málaga,Sevilla,Huesca,Teruel,Zaragoza,Asturias,Illes Balears,Las Palmas,Santa Cruz de Tenerife,Cantabria,Ávila,Burgos,León,Palencia,Salamanca,Segovia,Soria,Valladolid,Zamora,Albacete,Ciudad Real,Cuenca,Guadalajara,Toledo,Barcelona,Girona,Lleida,Tarragona,Alicante/Alacant,Castellón/Castelló,Valencia/València,Badajoz,Cáceres,A Coruña,Lugo,Ourense,Pontevedra,Madrid,Murcia,Navarra,Araba/Álava,Gipuzkoa,Bizkaia,La Rioja,Ceuta,Melilla
0,Año 2009,Primero,Presentada directamente por victima,40.0,140.0,11.0,111.0,14.0,17.0,100.0,126.0,7.0,30.0,19.0,9.0,137.0,11.0,130.0,44.0,7.0,24.0,20.0,2.0,120.0,3.0,0.0,9.0,11.0,21.0,27.0,3.0,57.0,103.0,173.0,37.0,15.0,212.0,57.0,9.0,327.0,70.0,107.0,17.0,15.0,19.0,83.0,252.0,58.0,5.0,9.0,24.0,34.0,51.0,0.0,2.0
1,Año 2009,Primero,Presentada directamente por familiares,1.0,17.0,0.0,0.0,0.0,0.0,4.0,8.0,4.0,0.0,1.0,1.0,17.0,3.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,0.0,0.0,2.0,3.0,0.0,0.0,1.0,0.0,3.0,1.0,0.0,2.0,11.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Año 2009,Primero,Atestados policiales - con denuncia victima,419.0,549.0,307.0,748.0,437.0,313.0,837.0,731.0,63.0,9.0,409.0,427.0,650.0,828.0,624.0,158.0,71.0,148.0,138.0,68.0,22.0,47.0,38.0,283.0,57.0,176.0,176.0,85.0,209.0,201.0,1929.0,378.0,202.0,271.0,1159.0,189.0,898.0,104.0,89.0,298.0,76.0,172.0,453.0,4089.0,958.0,187.0,173.0,230.0,397.0,100.0,57.0,61.0
3,Año 2009,Primero,Atestados policiales - con denuncia familiar,7.0,8.0,6.0,4.0,0.0,7.0,28.0,3.0,0.0,0.0,8.0,3.0,15.0,8.0,5.0,18.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,14.0,0.0,0.0,1.0,39.0,1.0,0.0,11.0,21.0,8.0,2.0,0.0,0.0,9.0,1.0,2.0,1.0,25.0,27.0,9.0,1.0,2.0,19.0,2.0,0.0,0.0
4,Año 2009,Primero,Atestados policiales - por intervención direct...,82.0,30.0,13.0,15.0,25.0,30.0,210.0,38.0,8.0,0.0,72.0,20.0,107.0,91.0,43.0,28.0,17.0,33.0,47.0,3.0,3.0,11.0,1.0,77.0,0.0,20.0,25.0,0.0,25.0,21.0,535.0,96.0,8.0,115.0,267.0,51.0,780.0,7.0,10.0,32.0,8.0,3.0,0.0,831.0,175.0,36.0,5.0,12.0,71.0,34.0,7.0,9.0


In [3]:
from suport import unif_col
denu.columns = unif_col(denu.columns)

In [4]:
from suport import lower_tildes
denu['trimestre'] = lower_tildes(denu['trimestre'])

In [5]:
denu.head()

,ano,trimestre,origen de la denuncia,almeria,cadiz,cordoba,granada,huelva,jaen,malaga,sevilla,huesca,teruel,zaragoza,asturias,illes balears,las palmas,santa cruz de tenerife,cantabria,avila,burgos,leon,palencia,salamanca,segovia,soria,valladolid,zamora,albacete,ciudad real,cuenca,guadalajara,toledo,barcelona,girona,lleida,tarragona,alicante/alacant,castellon/castello,valencia/valencia,badajoz,caceres,a coruna,lugo,ourense,pontevedra,madrid,murcia,navarra,araba/alava,gipuzkoa,bizkaia,la rioja,ceuta,melilla
0,Año 2009,primero,Presentada directamente por victima,40.0,140.0,11.0,111.0,14.0,17.0,100.0,126.0,7.0,30.0,19.0,9.0,137.0,11.0,130.0,44.0,7.0,24.0,20.0,2.0,120.0,3.0,0.0,9.0,11.0,21.0,27.0,3.0,57.0,103.0,173.0,37.0,15.0,212.0,57.0,9.0,327.0,70.0,107.0,17.0,15.0,19.0,83.0,252.0,58.0,5.0,9.0,24.0,34.0,51.0,0.0,2.0
1,Año 2009,primero,Presentada directamente por familiares,1.0,17.0,0.0,0.0,0.0,0.0,4.0,8.0,4.0,0.0,1.0,1.0,17.0,3.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,0.0,0.0,2.0,3.0,0.0,0.0,1.0,0.0,3.0,1.0,0.0,2.0,11.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Año 2009,primero,Atestados policiales - con denuncia victima,419.0,549.0,307.0,748.0,437.0,313.0,837.0,731.0,63.0,9.0,409.0,427.0,650.0,828.0,624.0,158.0,71.0,148.0,138.0,68.0,22.0,47.0,38.0,283.0,57.0,176.0,176.0,85.0,209.0,201.0,1929.0,378.0,202.0,271.0,1159.0,189.0,898.0,104.0,89.0,298.0,76.0,172.0,453.0,4089.0,958.0,187.0,173.0,230.0,397.0,100.0,57.0,61.0
3,Año 2009,primero,Atestados policiales - con denuncia familiar,7.0,8.0,6.0,4.0,0.0,7.0,28.0,3.0,0.0,0.0,8.0,3.0,15.0,8.0,5.0,18.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,14.0,0.0,0.0,1.0,39.0,1.0,0.0,11.0,21.0,8.0,2.0,0.0,0.0,9.0,1.0,2.0,1.0,25.0,27.0,9.0,1.0,2.0,19.0,2.0,0.0,0.0
4,Año 2009,primero,Atestados policiales - por intervención direct...,82.0,30.0,13.0,15.0,25.0,30.0,210.0,38.0,8.0,0.0,72.0,20.0,107.0,91.0,43.0,28.0,17.0,33.0,47.0,3.0,3.0,11.0,1.0,77.0,0.0,20.0,25.0,0.0,25.0,21.0,535.0,96.0,8.0,115.0,267.0,51.0,780.0,7.0,10.0,32.0,8.0,3.0,0.0,831.0,175.0,36.0,5.0,12.0,71.0,34.0,7.0,9.0


In [6]:
denu.rename(columns={'ano': 'año'}, inplace=True)
from suport import clean_year
denu.año = clean_year(denu.año)

In [7]:
denu.rename(columns={'origen de la denuncia': 'origen_denuncia'}, inplace=True)
denu['origen_denuncia'] = lower_tildes(denu['origen_denuncia'])

In [8]:
denu['origen_denuncia'].unique()

array(['presentada directamente por victima',
       'presentada directamente por familiares',
       'atestados policiales - con denuncia victima',
       'atestados policiales - con denuncia familiar',
       'atestados policiales - por intervencion directa policial',
       'parte de lesiones', 'servicios asistencia-terceros  en general'],
      dtype=object)

In [9]:
origen = {'presentada directamente por victima': 'victima',
          'presentada directamente por familiares': 'familiar',
          'atestados policiales - con denuncia victima': 'policia_victima',
          'atestados policiales - con denuncia familiar': 'policia_familiar',
          'atestados policiales - por intervencion directa policial': 'intervencion_policial_directa',
          'parte de lesiones': 'parte_lesiones',
          'servicios asistencia-terceros  en general': 'asistencia_terceros'}

denu['origen_denuncia'] = denu['origen_denuncia'].map(origen)

In [10]:
denu.head()

,año,trimestre,origen_denuncia,almeria,cadiz,cordoba,granada,huelva,jaen,malaga,sevilla,huesca,teruel,zaragoza,asturias,illes balears,las palmas,santa cruz de tenerife,cantabria,avila,burgos,leon,palencia,salamanca,segovia,soria,valladolid,zamora,albacete,ciudad real,cuenca,guadalajara,toledo,barcelona,girona,lleida,tarragona,alicante/alacant,castellon/castello,valencia/valencia,badajoz,caceres,a coruna,lugo,ourense,pontevedra,madrid,murcia,navarra,araba/alava,gipuzkoa,bizkaia,la rioja,ceuta,melilla
0,2009,primero,victima,40.0,140.0,11.0,111.0,14.0,17.0,100.0,126.0,7.0,30.0,19.0,9.0,137.0,11.0,130.0,44.0,7.0,24.0,20.0,2.0,120.0,3.0,0.0,9.0,11.0,21.0,27.0,3.0,57.0,103.0,173.0,37.0,15.0,212.0,57.0,9.0,327.0,70.0,107.0,17.0,15.0,19.0,83.0,252.0,58.0,5.0,9.0,24.0,34.0,51.0,0.0,2.0
1,2009,primero,familiar,1.0,17.0,0.0,0.0,0.0,0.0,4.0,8.0,4.0,0.0,1.0,1.0,17.0,3.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,0.0,0.0,2.0,3.0,0.0,0.0,1.0,0.0,3.0,1.0,0.0,2.0,11.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2009,primero,policia_victima,419.0,549.0,307.0,748.0,437.0,313.0,837.0,731.0,63.0,9.0,409.0,427.0,650.0,828.0,624.0,158.0,71.0,148.0,138.0,68.0,22.0,47.0,38.0,283.0,57.0,176.0,176.0,85.0,209.0,201.0,1929.0,378.0,202.0,271.0,1159.0,189.0,898.0,104.0,89.0,298.0,76.0,172.0,453.0,4089.0,958.0,187.0,173.0,230.0,397.0,100.0,57.0,61.0
3,2009,primero,policia_familiar,7.0,8.0,6.0,4.0,0.0,7.0,28.0,3.0,0.0,0.0,8.0,3.0,15.0,8.0,5.0,18.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,14.0,0.0,0.0,1.0,39.0,1.0,0.0,11.0,21.0,8.0,2.0,0.0,0.0,9.0,1.0,2.0,1.0,25.0,27.0,9.0,1.0,2.0,19.0,2.0,0.0,0.0
4,2009,primero,intervencion_policial_directa,82.0,30.0,13.0,15.0,25.0,30.0,210.0,38.0,8.0,0.0,72.0,20.0,107.0,91.0,43.0,28.0,17.0,33.0,47.0,3.0,3.0,11.0,1.0,77.0,0.0,20.0,25.0,0.0,25.0,21.0,535.0,96.0,8.0,115.0,267.0,51.0,780.0,7.0,10.0,32.0,8.0,3.0,0.0,831.0,175.0,36.0,5.0,12.0,71.0,34.0,7.0,9.0


In [11]:
# Guardamos el dataframe en un archivo csv:

denu.to_csv('../data_clean/portal_estadistico_vio_gen/denuncias.csv', index=False)

### LIMPIEZA ARCHIVO LLAMADAS 016:

In [12]:
llam = pd.read_csv('../data/llamadas.csv')
llam.head()

,Año,Trimestre,Llamante,Almería,Cádiz,Córdoba,Granada,Huelva,Jaén,Málaga,Sevilla,Huesca,Teruel,Zaragoza,Asturias,Illes Balears,Las Palmas,Santa Cruz de Tenerife,Cantabria,Ávila,Burgos,León,Palencia,Salamanca,Segovia,Soria,Valladolid,Zamora,Albacete,Ciudad Real,Cuenca,Guadalajara,Toledo,Barcelona,Girona,Lleida,Tarragona,Alicante/Alacant,Castellón/Castelló,Valencia/València,Badajoz,Cáceres,A Coruña,Lugo,Ourense,Pontevedra,Madrid,Murcia,Navarra,Araba/Álava,Bizkaia,Gipuzkoa,La Rioja,Ceuta,Melilla,No consta
0,Año 2007,Tercero,Usuaria,3.0,2.0,7.0,6.0,4.0,NaN,8.0,16.0,1.0,NaN,4.0,5.0,6.0,20.0,6.0,8.0,3.0,NaN,2.0,2.0,1.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,1.0,1.0,27.0,3.0,1.0,4.0,8.0,2.0,4.0,3.0,4.0,5.0,NaN,2.0,2.0,99.0,6.0,4.0,1.0,2.0,1.0,NaN,1.0,1.0,16.0
1,Año 2007,Tercero,Familiares/Personas allegadas,NaN,NaN,3.0,1.0,2.0,NaN,5.0,9.0,NaN,NaN,NaN,2.0,1.0,3.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,1.0,5.0,NaN,NaN,1.0,5.0,NaN,4.0,2.0,1.0,1.0,NaN,NaN,1.0,30.0,1.0,NaN,NaN,2.0,NaN,NaN,NaN,1.0,3.0
2,Año 2007,Tercero,Otros,NaN,NaN,NaN,1.0,2.0,2.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,4.0,2.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,20.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,3.0
3,Año 2007,Cuarto,Usuaria,150.0,187.0,155.0,250.0,82.0,131.0,357.0,608.0,37.0,40.0,152.0,248.0,309.0,365.0,298.0,177.0,60.0,65.0,128.0,44.0,80.0,48.0,25.0,159.0,36.0,50.0,93.0,34.0,35.0,180.0,1229.0,90.0,83.0,134.0,498.0,102.0,592.0,160.0,78.0,317.0,56.0,64.0,157.0,3368.0,386.0,123.0,62.0,195.0,76.0,49.0,22.0,18.0,123.0
4,Año 2007,Cuarto,Familiares/Personas allegadas,27.0,44.0,26.0,41.0,17.0,17.0,78.0,135.0,8.0,2.0,35.0,51.0,43.0,71.0,49.0,23.0,12.0,10.0,16.0,8.0,25.0,1.0,8.0,22.0,2.0,13.0,23.0,7.0,9.0,36.0,247.0,24.0,15.0,21.0,75.0,21.0,125.0,26.0,19.0,45.0,9.0,1.0,33.0,604.0,61.0,32.0,16.0,38.0,19.0,12.0,2.0,3.0,28.0


In [22]:
llam.columns = unif_col(llam.columns)

In [14]:
llam['trimestre'] = lower_tildes(llam['trimestre'])
llam.llamante = lower_tildes(llam.llamante)

In [24]:
llam.llamante.unique()

array(['usuaria', 'familiares/personas allegadas', 'otros', 'desconocido',
       'no consta'], dtype=object)

In [25]:
llam.columns

Index(['ano', 'trimestre', 'llamante', 'almeria', 'cadiz', 'cordoba',
       'granada', 'huelva', 'jaen', 'malaga', 'sevilla', 'huesca', 'teruel',
       'zaragoza', 'asturias', 'illes balears', 'las palmas',
       'santa cruz de tenerife', 'cantabria', 'avila', 'burgos', 'leon',
       'palencia', 'salamanca', 'segovia', 'soria', 'valladolid', 'zamora',
       'albacete', 'ciudad real', 'cuenca', 'guadalajara', 'toledo',
       'barcelona', 'girona', 'lleida', 'tarragona', 'alicante/alacant',
       'castellon/castello', 'valencia/valencia', 'badajoz', 'caceres',
       'a coruna', 'lugo', 'ourense', 'pontevedra', 'madrid', 'murcia',
       'navarra', 'araba/alava', 'bizkaia', 'gipuzkoa', 'la rioja', 'ceuta',
       'melilla', 'no consta'],
      dtype='object')

In [28]:
quien_llama = {'usuaria': 'usuaria',
               'familiares/personas allegadas': 'familiar',
               'otros': 'desconocido',
               'desconocido': 'desconocido',
               'no Consta': 'desconocido'}

llam['llamante'] = llam['llamante'].map(quien_llama)

In [33]:
llam.rename(columns={'ano': 'año'}, inplace=True)
llam.año = clean_year(llam.año)

In [34]:
llam.fillna(0, inplace = True)

llam.head()

,año,trimestre,llamante,almeria,cadiz,cordoba,granada,huelva,jaen,malaga,sevilla,huesca,teruel,zaragoza,asturias,illes balears,las palmas,santa cruz de tenerife,cantabria,avila,burgos,leon,palencia,salamanca,segovia,soria,valladolid,zamora,albacete,ciudad real,cuenca,guadalajara,toledo,barcelona,girona,lleida,tarragona,alicante/alacant,castellon/castello,valencia/valencia,badajoz,caceres,a coruna,lugo,ourense,pontevedra,madrid,murcia,navarra,araba/alava,bizkaia,gipuzkoa,la rioja,ceuta,melilla,no consta
0,2007,tercero,usuaria,3.0,2.0,7.0,6.0,4.0,0.0,8.0,16.0,1.0,0.0,4.0,5.0,6.0,20.0,6.0,8.0,3.0,0.0,2.0,2.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,27.0,3.0,1.0,4.0,8.0,2.0,4.0,3.0,4.0,5.0,0.0,2.0,2.0,99.0,6.0,4.0,1.0,2.0,1.0,0.0,1.0,1.0,16.0
1,2007,tercero,familiar,0.0,0.0,3.0,1.0,2.0,0.0,5.0,9.0,0.0,0.0,0.0,2.0,1.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,1.0,5.0,0.0,4.0,2.0,1.0,1.0,0.0,0.0,1.0,30.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,3.0
2,2007,tercero,desconocido,0.0,0.0,0.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,4.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,20.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,3.0
3,2007,cuarto,usuaria,150.0,187.0,155.0,250.0,82.0,131.0,357.0,608.0,37.0,40.0,152.0,248.0,309.0,365.0,298.0,177.0,60.0,65.0,128.0,44.0,80.0,48.0,25.0,159.0,36.0,50.0,93.0,34.0,35.0,180.0,1229.0,90.0,83.0,134.0,498.0,102.0,592.0,160.0,78.0,317.0,56.0,64.0,157.0,3368.0,386.0,123.0,62.0,195.0,76.0,49.0,22.0,18.0,123.0
4,2007,cuarto,familiar,27.0,44.0,26.0,41.0,17.0,17.0,78.0,135.0,8.0,2.0,35.0,51.0,43.0,71.0,49.0,23.0,12.0,10.0,16.0,8.0,25.0,1.0,8.0,22.0,2.0,13.0,23.0,7.0,9.0,36.0,247.0,24.0,15.0,21.0,75.0,21.0,125.0,26.0,19.0,45.0,9.0,1.0,33.0,604.0,61.0,32.0,16.0,38.0,19.0,12.0,2.0,3.0,28.0


In [35]:
llam.to_csv('../data_clean/portal_estadistico_vio_gen/llamadas_016.csv', index=False)